In [1]:
!pip install spacy -q
!pip install spacy spacy-langdetect -q
!pip install --upgrade pandas -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autovizwidget 0.21.0 requires pandas<2.0.0,>=0.20.1, but you have pandas 2.2.1 which is incompatible.
hdijupyterutils 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.2.1 which is incompatible.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.2.1 which is incompatible.


In [2]:
import pandas as pd
print(pd.__version__)
import json
from tqdm import tqdm
from datetime import datetime
import requests
import os

import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector
!python -m spacy download en_core_web_sm -q

2.2.1
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# Load raw data

data on my github:
https://github.com/Yung-Chun/WTO-News-and-Events-Archive-Crawler/blob/main/WTO_data_article/all_article_content.json

In [3]:
# URL of the JSON file
url = "https://raw.githubusercontent.com/Yung-Chun/WTO-News-and-Events-Archive-Crawler/main/WTO_data_article/all_article_content.json"

try:
    # Fetch the data from the URL
    response = requests.get(url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        
        # Parse JSON data
        all_article_content = response.json()
        print("Seccessfully fetched data. Status code:", response.status_code)
        
    else:
        print("Failed to fetch data. Status code:", response.status_code)

except requests.exceptions.RequestException as e:
    print("Error fetching data:", e)

Seccessfully fetched data. Status code: 200


In [4]:
len(all_article_content)

8680

In [5]:
def check_date_format(date):
    try:
        datetime.strptime(date, "%d %B %Y")
        return True
    except:
        return False

In [6]:
def combine_content_and_abstract(url):
    content = all_article_content[url]['content']
    abstract = all_article_content[url]['abstract']
     
    if abstract == None:
        combined_content = content
    elif content == None:
        combined_content = abstract
    else:
        combined_content = ' '.join([abstract, content])
    
    return combined_content

In [7]:
def get_lang_detector(nlp, name):
    return LanguageDetector()

nlp = spacy.load("en_core_web_sm")
Language.factory("language_detector", func=get_lang_detector)
nlp.add_pipe('language_detector', last=True)

def language_detection(doc):
    doc = nlp(doc)
    return doc._.language['language']=='en'

## Get English docs, check date  format, and save

In [11]:
if not os.path.exists('training_data'):
    os.makedirs('training_data')
    print('Created folder named training_data')
    
else:
    print('Folder named training_data exists')

Folder named training_data exists


In [ ]:
wto_eng_docs = {}
idx = 0

for url in tqdm(all_article_content.keys(), desc='Processing'):
    try:
        date = all_article_content[url]['date']
        doc = combine_content_and_abstract(url)
        
        if check_date_format(date) and language_detection(doc):
            wto_eng_docs.update({url: {'date': date, 'doc': doc}})
    except:
        pass
    
    if idx % 100 == 0:
        with open('training_data/wto_eng_docs.json', 'w') as f:
            json.dump(wto_eng_docs, f)
    idx += 1

with open('training_data/wto_eng_docs.json', 'w') as f:
    json.dump(wto_eng_docs, f)

print(f'length of wto_eng_docs {len(wto_eng_docs)}')

Processing:  97%|█████████▋| 8442/8680 [23:03<00:17, 13.73it/s]  